<a href="https://colab.research.google.com/github/Athugodage/Summarization_LEA/blob/main/newest_pegasus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 16.4 MB/s eta 0:00:00


In [4]:
!pip install transformers rouge_score summa datasets Sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 93.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [5]:
from datasets import load_dataset

dataset = load_dataset('ccdv/WCEP-10')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset wcep-10 downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___wcep-10/roberta/1.0.0/d4660eb971bdf9fa722e74ae48eda378ce9554fac9d3c19714486cff2860f11e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
from abc import ABC, abstractmethod


class AbstractModel(ABC):
    def __init__(
            self,
            max_target_length=64,
            max_source_length=1024,
            save_path="./"
    ):
        self.save_path = save_path
        self.max_target_length = max_target_length
        self.max_source_length = max_source_length

    @abstractmethod
    def train(self, train_dataset, val_dataset=None):
        raise NotImplementedError()

    @abstractmethod
    def predict(self, test_dataset):
        raise NotImplementedError()

In [7]:
from tqdm import tqdm
from summa.summarizer import summarize
from transformers import AutoTokenizer


class TextRankModel(AbstractModel):
    def __init__(
            self,
            max_target_length,
            max_source_length,
            save_path="./textrank-model/",
            tokenizer_checkpoint='google/pegasus-large'
    ):
        super().__init__(
            max_target_length=max_target_length,
            max_source_length=max_source_length,
            save_path=save_path
        )

        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
        self.test_dataset = None

    def train(self, train_dataset, val_dataset=None):
        pass

    def predict(self, test_dataset):
        model_inputs = self.tokenizer(
            test_dataset['document'],
            max_length=self.max_source_length,
            padding='max_length',
            truncation=True
        )
        articles = self.tokenizer.batch_decode(model_inputs['input_ids'], skip_special_tokens=True)

        model_inputs = self.tokenizer(test_dataset['summary'])
        summaries = self.tokenizer.batch_decode(model_inputs['input_ids'], skip_special_tokens=True)

        self.test_dataset = Dataset.from_dict({'document': articles, 'summary': summaries})

        if self.max_source_length is not None:
            return [summarize(article, words=self.max_target_length) for article in
                    tqdm(self.test_dataset['document'])]
        else:
            return [summarize(article, words=self.max_target_length) for article in tqdm(self.test_dataset['articles'])]

In [8]:
from datasets import load_metric
from transformers import DataCollatorForLanguageModeling, AutoModelForSeq2SeqLM, IntervalStrategy, Trainer, TrainingArguments
import numpy as np





class PegasusModel(AbstractModel):
    def __init__(
            self,
            max_target_length=121,
            max_source_length=1024,
            save_path="pegasusTextrankedonWCEP",
            model_checkpoint='google/pegasus-large',
            tokenizer_checkpoint='google/pegasus-large',
            epochs=1,
            batch_size=1
    ):
        super().__init__(
            max_target_length=max_target_length,
            max_source_length=max_source_length,
            save_path=save_path
        )
        self.batch_size = batch_size
        self.epochs = epochs

        self.train_dataset = None
        self.test_dataset = None
        self.val_dataset = None

        self.rouge_score = load_metric("rouge")
        self.trainer = None
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
        self.data_collator = DataCollatorForLanguageModeling(self.tokenizer, mlm_probability=0.15)

        self.args = TrainingArguments(
            output_dir=self.save_path,
            evaluation_strategy=IntervalStrategy.EPOCH,
            learning_rate=5.6e-6,
            per_device_train_batch_size=self.batch_size,
            per_device_eval_batch_size=self.batch_size,
            weight_decay=0.01,
            save_total_limit=1,
            gradient_accumulation_steps=10,
            eval_accumulation_steps=2,
            num_train_epochs=self.epochs,
            logging_strategy=IntervalStrategy.EPOCH,
            push_to_hub=True,
        )

    def __preprocess_function__(self, batch):
        # tokenize the inputs and labels
        inputs = self.tokenizer(
            batch["document"],
            padding="max_length",
            truncation=True,
            max_length=self.max_source_length,
        )
        outputs = self.tokenizer(
            batch["summary"],
            padding="max_length",
            truncation=True,
            max_length=self.max_target_length,
        )

        batch["input_ids"] = inputs.input_ids
        batch["attention_mask"] = inputs.attention_mask

        # create 0 global_attention_mask lists
        batch["global_attention_mask"] = len(batch["input_ids"]) * [
            [0 for _ in range(len(batch["input_ids"][0]))]
        ]

        # since above lists are references, the following line changes the 0 index for all samples
        batch["global_attention_mask"][0][0] = 1
        batch["labels"] = outputs.input_ids

        # We have to make sure that the PAD token is ignored
        batch["labels"] = [
            [-100 if token == self.tokenizer.pad_token_id else token for token in labels]
            for labels in batch["labels"]
        ]

        return batch


    def train(self, train_dataset, val_dataset=None):
        self.train_dataset = train_dataset.map(self.__preprocess_function__, batched=True, remove_columns=['document', 'summary'])
        
        if val_dataset is not None:
            self.val_dataset = val_dataset.map(self.__preprocess_function__, batched=True, remove_columns=['document', 'summary'])
        
        else:
            self.val_dataset = None

        self.trainer = Trainer(
            self.model,
            self.args,
            train_dataset=self.train_dataset,
            eval_dataset=self.val_dataset,
            data_collator=self.data_collator,
            tokenizer=self.tokenizer,
            compute_metrics=self.compute_metrics,
        )

        self.trainer.train()

    def predict(self, test_dataset):
        self.test_dataset = test_dataset.map(self.__preprocess_function__, batched=True, remove_columns=['document', 'summary'])

        if self.trainer is None:
            self.trainer = Trainer(
                self.model,
                self.args,
                data_collator=self.data_collator,
                tokenizer=self.tokenizer,
                compute_metrics=self.compute_metrics,
            )

        return self.trainer.predict(test_dataset)


    def postprocess_text(self, preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels


    def compute_metrics(self, eval_pred):
        preds, labels = eval_pred
        if isinstance(preds, tuple):
            preds = preds[0]

        decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)

        rouge_result = self.rouge_score.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=True
        )

        rouge_result = {key: round(value.mid.fmeasure * 100, 4) for key, value in rouge_result.items()}


        return rouge_result

In [9]:
'''save_path="longformer-textrank/"
model_checkpoint='allenai/longformer-base-4096'
tokenizer_checkpoint='allenai/longformer-base-4096'
epochs=1
batch_size=1

ideal_model = LongformerModel(
            max_target_length=100,
            max_source_length=1000,
            model_checkpoint=model_checkpoint,
            tokenizer_checkpoint=tokenizer_checkpoint,
            epochs=epochs,
            batch_size=batch_size
        )

textranker = TextRankModel(max_target_length=100, max_source_length=1000)
ideal_model.train(train_dataset=dataset['train'], val_dataset=dataset['validation'])'''

'save_path="longformer-textrank/"\nmodel_checkpoint=\'allenai/longformer-base-4096\'\ntokenizer_checkpoint=\'allenai/longformer-base-4096\'\nepochs=1\nbatch_size=1\n\nideal_model = LongformerModel(\n            max_target_length=100,\n            max_source_length=1000,\n            model_checkpoint=model_checkpoint,\n            tokenizer_checkpoint=tokenizer_checkpoint,\n            epochs=epochs,\n            batch_size=batch_size\n        )\n\ntextranker = TextRankModel(max_target_length=100, max_source_length=1000)\nideal_model.train(train_dataset=dataset[\'train\'], val_dataset=dataset[\'validation\'])'

In [10]:
from transformers import AutoTokenizer, pipeline
from datasets import Dataset

class PegasusTextRankModel():
    def __init__(
            self,
            save_path="./longformer-textrank/",
            model_checkpoint='google/pegasus-large',
            tokenizer_checkpoint='google/pegasus-large',
            epochs=1,
            batch_size=1
    ):

        self.epochs = epochs
        self.batch_size = batch_size

        self.pegasus = PegasusModel(
            max_target_length=1024,
            max_source_length=121,
            model_checkpoint=model_checkpoint,
            tokenizer_checkpoint=tokenizer_checkpoint,
            epochs=epochs,
            batch_size=batch_size
        )
        self.textrank = TextRankModel(max_target_length=100, max_source_length=1000)

    def train(self, train_dataset, val_dataset=None):
        self.pegasus.train(train_dataset, val_dataset)


    def predict(self, test_dataset):
        test = test_dataset.map(self.__preprocess_function__, batched=True, remove_columns=['document', 'summary', 'id'])
        

        bart_predictions = [self.pegasus.tokenizer.decode(prediction, skip_special_tokens=True) for prediction in
                            self.pegasus.predict(test).predictions]
        textrank_predictions = self.textrank.predict(test_dataset)

        test = Dataset.from_dict(
            {'document': [bart_predictions[i] + "  " + textrank_predictions[i] for i in range(len(test_dataset))],
             'summary': test['summary']})

        return self.pegasus.predict(test)

In [11]:
l = PegasusTextRankModel()

<ipython-input-8-3ad39fe2a0fe>:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.rouge_score = load_metric("rouge")


In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 8158
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 1020
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 1022
    })
})

In [ ]:
l.train(train_dataset=dataset['train'], val_dataset=dataset['validation'])

Map:   0%|          | 0/8158 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

Cloning https://huggingface.co/marcus2000/pegasusTextrankedonWCEP into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
l.longformer.predict(test_dataset=dataset['test'])

In [ ]:
l.longformer.compute_metrics()